In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ferbi\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 20*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [3]:
for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
    for i in [1, 2, 3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20240326225421
0	Validation losses: 0.4254, 15.6827, 16.1081	Best Loss: 16.1081 (0)	Accuracy: 84.02%
1	Validation losses: 0.3713, 10.2709, 10.6422	Best Loss: 10.6422 (0)	Accuracy: 87.14%
2	Validation losses: 0.3815, 5.2372, 5.6187	Best Loss: 5.6187 (0)	Accuracy: 86.24%
3	Validation losses: 0.4237, 2.1196, 2.5433	Best Loss: 2.5433 (0)	Accuracy: 84.74%
4	Validation losses: 0.3887, 0.7708, 1.1595	Best Loss: 1.1595 (0)	Accuracy: 85.22%
5	Validation losses: 0.4922, 0.3470, 0.8392	Best Loss: 0.8392 (0)	Accuracy: 82.16%
6	Validation losses: 0.3595, 0.2441, 0.6035	Best Loss: 0.6035 (0)	Accuracy: 87.24%
7	Validation losses: 0.4961, 0.2174, 0.7136	Best Loss: 0.6035 (1)	Accuracy: 80.84%
8	Validation losses: 0.4178, 0.2087, 0.6265	Best Loss: 0.6035 (2)	Accuracy: 85.14%
9	Validation losses: 0.5417, 0.2080, 0.7497	Best Loss: 0.6035 (3)	Accuracy: 80.62%
10	Validation losses: 0.4592, 0.2113, 0.6704	Best Loss: 0.6035 (4)	Accuracy: 83.22%
11	Validation losses: 0

3	Validation losses: 0.3879, 3.0483, 3.4362	Best Loss: 3.4362 (0)	Accuracy: 86.00%
4	Validation losses: 0.3633, 1.5079, 1.8712	Best Loss: 1.8712 (0)	Accuracy: 86.30%
5	Validation losses: 0.5072, 0.7015, 1.2087	Best Loss: 1.2087 (0)	Accuracy: 82.12%
6	Validation losses: 0.3444, 0.3596, 0.7041	Best Loss: 0.7041 (0)	Accuracy: 87.66%
7	Validation losses: 0.4581, 0.2367, 0.6947	Best Loss: 0.6947 (0)	Accuracy: 82.96%
8	Validation losses: 0.3863, 0.2012, 0.5875	Best Loss: 0.5875 (0)	Accuracy: 86.20%
9	Validation losses: 0.5185, 0.1901, 0.7086	Best Loss: 0.5875 (1)	Accuracy: 82.54%
10	Validation losses: 0.3865, 0.1858, 0.5723	Best Loss: 0.5723 (0)	Accuracy: 85.86%
11	Validation losses: 0.3968, 0.1866, 0.5833	Best Loss: 0.5723 (1)	Accuracy: 85.74%
12	Validation losses: 0.8725, 0.1838, 1.0563	Best Loss: 0.5723 (2)	Accuracy: 73.96%
13	Validation losses: 0.4537, 0.1774, 0.6312	Best Loss: 0.5723 (3)	Accuracy: 83.12%
14	Validation losses: 0.5516, 0.1975, 0.7491	Best Loss: 0.5723 (4)	Accuracy: 79.54%

36	Validation losses: 0.3567, 0.0862, 0.4429	Best Loss: 0.4429 (0)	Accuracy: 86.94%
37	Validation losses: 0.3574, 0.0854, 0.4428	Best Loss: 0.4428 (0)	Accuracy: 87.28%
38	Validation losses: 0.3593, 0.0846, 0.4439	Best Loss: 0.4428 (1)	Accuracy: 87.18%
39	Validation losses: 0.3534, 0.0839, 0.4373	Best Loss: 0.4373 (0)	Accuracy: 87.50%
40	Validation losses: 0.3487, 0.0836, 0.4323	Best Loss: 0.4323 (0)	Accuracy: 87.62%
41	Validation losses: 0.3547, 0.0834, 0.4380	Best Loss: 0.4323 (1)	Accuracy: 87.54%
42	Validation losses: 0.3586, 0.0834, 0.4419	Best Loss: 0.4323 (2)	Accuracy: 87.42%
43	Validation losses: 0.3593, 0.0838, 0.4431	Best Loss: 0.4323 (3)	Accuracy: 87.18%
44	Validation losses: 0.3553, 0.0847, 0.4400	Best Loss: 0.4323 (4)	Accuracy: 87.50%
45	Validation losses: 0.3615, 0.0851, 0.4467	Best Loss: 0.4323 (5)	Accuracy: 86.96%
46	Validation losses: 0.3654, 0.0863, 0.4517	Best Loss: 0.4323 (6)	Accuracy: 87.14%
47	Validation losses: 0.3598, 0.0872, 0.4470	Best Loss: 0.4323 (7)	Accuracy:

1	Validation losses: 0.3588, 5.9470, 6.3058	Best Loss: 6.3058 (0)	Accuracy: 87.02%
2	Validation losses: 0.3504, 4.6040, 4.9544	Best Loss: 4.9544 (0)	Accuracy: 87.22%
3	Validation losses: 0.4043, 3.2394, 3.6437	Best Loss: 3.6437 (0)	Accuracy: 84.70%
4	Validation losses: 0.3365, 2.0777, 2.4142	Best Loss: 2.4142 (0)	Accuracy: 88.02%
5	Validation losses: 0.3702, 1.2305, 1.6007	Best Loss: 1.6007 (0)	Accuracy: 86.56%
6	Validation losses: 0.4347, 0.6977, 1.1324	Best Loss: 1.1324 (0)	Accuracy: 83.82%
7	Validation losses: 0.3892, 0.4066, 0.7958	Best Loss: 0.7958 (0)	Accuracy: 86.78%
8	Validation losses: 0.3661, 0.2672, 0.6333	Best Loss: 0.6333 (0)	Accuracy: 85.98%
9	Validation losses: 0.4227, 0.2066, 0.6293	Best Loss: 0.6293 (0)	Accuracy: 85.58%
10	Validation losses: 0.3969, 0.1836, 0.5805	Best Loss: 0.5805 (0)	Accuracy: 85.34%
11	Validation losses: 0.4793, 0.1762, 0.6555	Best Loss: 0.5805 (1)	Accuracy: 82.70%
12	Validation losses: 0.4156, 0.1722, 0.5878	Best Loss: 0.5805 (2)	Accuracy: 84.90%
1

34	Validation losses: 0.3448, 0.1263, 0.4711	Best Loss: 0.4711 (0)	Accuracy: 87.30%
35	Validation losses: 0.3673, 0.1242, 0.4914	Best Loss: 0.4711 (1)	Accuracy: 86.96%
36	Validation losses: 0.3433, 0.1230, 0.4663	Best Loss: 0.4663 (0)	Accuracy: 87.52%
37	Validation losses: 0.3401, 0.1221, 0.4622	Best Loss: 0.4622 (0)	Accuracy: 87.94%
38	Validation losses: 0.3396, 0.1209, 0.4604	Best Loss: 0.4604 (0)	Accuracy: 87.94%
39	Validation losses: 0.3326, 0.1200, 0.4526	Best Loss: 0.4526 (0)	Accuracy: 88.06%
40	Validation losses: 0.3238, 0.1196, 0.4433	Best Loss: 0.4433 (0)	Accuracy: 88.42%
41	Validation losses: 0.3316, 0.1192, 0.4508	Best Loss: 0.4433 (1)	Accuracy: 88.08%
42	Validation losses: 0.3291, 0.1191, 0.4482	Best Loss: 0.4433 (2)	Accuracy: 88.24%
43	Validation losses: 0.3433, 0.1195, 0.4628	Best Loss: 0.4433 (3)	Accuracy: 87.60%
44	Validation losses: 0.3389, 0.1200, 0.4589	Best Loss: 0.4433 (4)	Accuracy: 88.06%
45	Validation losses: 0.3394, 0.1206, 0.4599	Best Loss: 0.4433 (5)	Accuracy:

...

13	Validation losses: 0.3222, 0.1210, 0.4432	Best Loss: 0.4132 (5)	Accuracy: 88.26%
14	Validation losses: 0.3228, 0.1215, 0.4443	Best Loss: 0.4132 (6)	Accuracy: 88.44%
15	Validation losses: 0.3230, 0.1220, 0.4450	Best Loss: 0.4132 (7)	Accuracy: 88.78%
16	Validation losses: 0.3150, 0.1227, 0.4377	Best Loss: 0.4132 (8)	Accuracy: 88.90%
17	Validation losses: 0.3201, 0.1235, 0.4436	Best Loss: 0.4132 (9)	Accuracy: 89.40%
18	Validation losses: 0.3560, 0.1241, 0.4801	Best Loss: 0.4132 (10)	Accuracy: 88.40%
19	Validation losses: 0.3279, 0.1249, 0.4528	Best Loss: 0.4132 (11)	Accuracy: 89.08%
20	Validation losses: 0.3685, 0.1258, 0.4943	Best Loss: 0.4132 (12)	Accuracy: 87.34%
21	Validation losses: 0.3324, 0.1261, 0.4585	Best Loss: 0.4132 (13)	Accuracy: 88.76%
22	Validation losses: 0.3810, 0.1260, 0.5070	Best Loss: 0.4132 (14)	Accuracy: 88.68%
23	Validation losses: 0.3313, 0.1257, 0.4570	Best Loss: 0.4132 (15)	Accuracy: 88.90%
24	Validation losses: 0.3418, 0.1251, 0.4669	Best Loss: 0.4132 (16)	Ac

12	Validation losses: 0.3303, 0.0795, 0.4098	Best Loss: 0.3769 (4)	Accuracy: 88.30%
13	Validation losses: 0.3160, 0.0802, 0.3962	Best Loss: 0.3769 (5)	Accuracy: 88.72%
14	Validation losses: 0.3257, 0.0810, 0.4067	Best Loss: 0.3769 (6)	Accuracy: 87.86%
15	Validation losses: 0.3364, 0.0818, 0.4182	Best Loss: 0.3769 (7)	Accuracy: 88.46%
16	Validation losses: 0.3251, 0.0829, 0.4080	Best Loss: 0.3769 (8)	Accuracy: 88.50%
17	Validation losses: 0.3031, 0.0839, 0.3871	Best Loss: 0.3769 (9)	Accuracy: 89.44%
18	Validation losses: 0.3352, 0.0850, 0.4202	Best Loss: 0.3769 (10)	Accuracy: 88.76%
19	Validation losses: 0.3641, 0.0862, 0.4503	Best Loss: 0.3769 (11)	Accuracy: 87.98%
20	Validation losses: 0.3897, 0.0874, 0.4771	Best Loss: 0.3769 (12)	Accuracy: 86.64%
21	Validation losses: 0.3796, 0.0884, 0.4681	Best Loss: 0.3769 (13)	Accuracy: 87.86%
22	Validation losses: 0.3859, 0.0890, 0.4750	Best Loss: 0.3769 (14)	Accuracy: 88.08%
23	Validation losses: 0.3635, 0.0894, 0.4529	Best Loss: 0.3769 (15)	Acc

5	Validation losses: 0.3385, 0.0483, 0.3868	Best Loss: 0.3655 (1)	Accuracy: 87.34%
6	Validation losses: 0.3369, 0.0486, 0.3855	Best Loss: 0.3655 (2)	Accuracy: 87.34%
7	Validation losses: 0.3643, 0.0489, 0.4132	Best Loss: 0.3655 (3)	Accuracy: 86.36%
8	Validation losses: 0.3000, 0.0493, 0.3493	Best Loss: 0.3493 (0)	Accuracy: 89.48%
9	Validation losses: 0.3201, 0.0498, 0.3699	Best Loss: 0.3493 (1)	Accuracy: 88.24%
10	Validation losses: 0.3296, 0.0503, 0.3800	Best Loss: 0.3493 (2)	Accuracy: 88.14%
11	Validation losses: 0.4303, 0.0509, 0.4811	Best Loss: 0.3493 (3)	Accuracy: 82.76%
12	Validation losses: 0.3380, 0.0515, 0.3895	Best Loss: 0.3493 (4)	Accuracy: 88.30%
13	Validation losses: 0.3206, 0.0523, 0.3728	Best Loss: 0.3493 (5)	Accuracy: 87.96%
14	Validation losses: 0.3034, 0.0530, 0.3564	Best Loss: 0.3493 (6)	Accuracy: 88.48%
15	Validation losses: 0.3143, 0.0551, 0.3694	Best Loss: 0.3493 (7)	Accuracy: 88.76%
16	Validation losses: 0.3423, 0.0560, 0.3982	Best Loss: 0.3493 (8)	Accuracy: 87.8

4	Validation losses: 0.3348, 0.0304, 0.3652	Best Loss: 0.3652 (0)	Accuracy: 87.24%
5	Validation losses: 0.3206, 0.0306, 0.3512	Best Loss: 0.3512 (0)	Accuracy: 87.92%
6	Validation losses: 0.3311, 0.0308, 0.3620	Best Loss: 0.3512 (1)	Accuracy: 87.94%
7	Validation losses: 0.3765, 0.0311, 0.4076	Best Loss: 0.3512 (2)	Accuracy: 86.62%
8	Validation losses: 0.2958, 0.0314, 0.3272	Best Loss: 0.3272 (0)	Accuracy: 89.62%
9	Validation losses: 0.3388, 0.0318, 0.3706	Best Loss: 0.3272 (1)	Accuracy: 88.00%
10	Validation losses: 0.3399, 0.0322, 0.3720	Best Loss: 0.3272 (2)	Accuracy: 87.72%
11	Validation losses: 0.3789, 0.0326, 0.4115	Best Loss: 0.3272 (3)	Accuracy: 85.38%
12	Validation losses: 0.3284, 0.0330, 0.3614	Best Loss: 0.3272 (4)	Accuracy: 88.66%
13	Validation losses: 0.3277, 0.0335, 0.3612	Best Loss: 0.3272 (5)	Accuracy: 88.92%
14	Validation losses: 0.3111, 0.0341, 0.3452	Best Loss: 0.3272 (6)	Accuracy: 88.64%
15	Validation losses: 0.3094, 0.0347, 0.3441	Best Loss: 0.3272 (7)	Accuracy: 88.82

1	Validation losses: 0.3543, 0.0190, 0.3733	Best Loss: 0.3733 (0)	Accuracy: 87.36%
2	Validation losses: 0.3468, 0.0190, 0.3659	Best Loss: 0.3659 (0)	Accuracy: 87.26%
3	Validation losses: 0.3927, 0.0191, 0.4118	Best Loss: 0.3659 (1)	Accuracy: 85.50%
4	Validation losses: 0.3250, 0.0192, 0.3442	Best Loss: 0.3442 (0)	Accuracy: 87.72%
5	Validation losses: 0.3255, 0.0194, 0.3449	Best Loss: 0.3442 (1)	Accuracy: 88.04%
6	Validation losses: 0.3393, 0.0195, 0.3588	Best Loss: 0.3442 (2)	Accuracy: 87.56%
7	Validation losses: 0.3937, 0.0197, 0.4134	Best Loss: 0.3442 (3)	Accuracy: 85.88%
8	Validation losses: 0.2950, 0.0199, 0.3149	Best Loss: 0.3149 (0)	Accuracy: 90.10%
9	Validation losses: 0.3127, 0.0202, 0.3329	Best Loss: 0.3149 (1)	Accuracy: 88.44%
10	Validation losses: 0.3159, 0.0204, 0.3363	Best Loss: 0.3149 (2)	Accuracy: 88.68%
11	Validation losses: 0.3691, 0.0207, 0.3899	Best Loss: 0.3149 (3)	Accuracy: 85.98%
12	Validation losses: 0.3354, 0.0211, 0.3564	Best Loss: 0.3149 (4)	Accuracy: 88.14%
1